In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
location = "properatti.csv"
data = pd.read_csv(location, sep=",", index_col=0)

Con todo lo que veo de los valores outliers de superficie_total_m2 es que los podemos imputar, si no vamos a trabajar con un modelo sobre precio de casas o store. Capaz como mencionaba el profe, los valores extremos nos sirvan para aplanar la tendencia en una regresion. 

Visualización del dataframe

In [ ]:
#

In [ ]:
pd.options.display.max_columns = None
#data.sample(10)
data.sample(4)

In [ ]:
data_limpia = data.copy()


In [ ]:
#Para ver la dispersion de los datos por tipo de propiedad creamos un grafico de caja de bigotes
data_limpia.boxplot(column= "surface_total_in_m2", by="property_type") 
plt.show()
#vemos que existen bastantes casos en casas, apartamentos que tienen datos extremos

In [ ]:
#graficamos con un grafico de bigotes como es la presencia de valores extremos en ambas variables 
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
sns.boxplot(data=data_limpia, x="property_type", y='surface_covered_in_m2', ax=axes[0])
sns.boxplot(data=data_limpia, x="property_type", y='surface_total_in_m2', ax=axes[1]);
#efectivamente hay valores extremos

In [ ]:
#revisamos la estadisticos de media, desviacion y rangos intercuartilicos
data_limpia[["surface_total_in_m2", "surface_covered_in_m2" ]].describe()

In [ ]:
#data_solo_superficie = data_limpia[["surface_total_in_m2", "surface_covered_in_m2" ]]
#data_solo_superficie

In [ ]:
#vamos a verificar cuales son los datos que se encuentran por encima del tercer quartil y buscamos los limites superiores e inferiores
#para determinar que registros pueden considerarse extremos
q1, q2, q3 = data_limpia["surface_total_in_m2"].quantile([0.25, 0.50, 0.75])
print("Q1:", q1,"\nq2:", q2,"\nq3:", q3 )

In [ ]:
#para surface_total_in_m2 hayamos el limite superior: aplicamos la formula de calcular el rango intercuartilico y luego el valor superior
iqr = q3-q1
ls = q3 + 1.5*iqr
print("El limite superior para considerar un outlier en superficie_total_m2 es:", ls)

In [ ]:
li = q1 - 1.5*iqr
print("El limite inferior para considerar un outlier en superficie_total_m2 es:", li)

In [ ]:
#aplicamos el metodo query para consultar que campos son los que cumplen con tener valores por encima del limite superior 
#datos_outliers_ls = 
data_limpia.query("surface_total_in_m2 > @ls").sample(4)
#vemos que estos valores extremos pertenecen a casas en zonas rurales por lo cuall tiene sentido sus superficie


In [ ]:
#los guardamos en una variable para luego extraer el indice
datos_outliers_ls = data_limpia.query("surface_total_in_m2 > @ls")
indice_total = datos_outliers_ls.index
indice_total

In [ ]:
data_limpia.groupby(["property_type"])["surface_total_in_m2"].describe()

In [ ]:

sup_total_ls = data_limpia.loc[indice_total,["surface_total_in_m2", "surface_covered_in_m2","property_type","place_name", "state_name"]]


In [ ]:
sup_total_ls.groupby(["property_type"])["surface_total_in_m2"].count()

In [ ]:
outlier_value = 10000
mask_consistencia_outlier_sup= sup_total_ls["surface_total_in_m2"] >= outlier_value
sup_total_ls[mask_consistencia_outlier_sup]

In [ ]:
datos_outliers_li= data_limpia.query("surface_total_in_m2 < @li")
datos_outliers_li
#no hay valores por debajo del limite inferior

In [ ]:
data_limpia_2 = data.copy()

In [ ]:
data_limpia_sin_ls = data_limpia_2.drop(axis=0, index=indice_total)
data_limpia_sin_ls.boxplot(column= ["surface_total_in_m2"], by="property_type") 
plt.show()


In [ ]:
#hasta aqui

Explorado los datos

In [ ]:
print(data.loc[filas_nan, ["price_aprox_local_currency"]])

In [ ]:
data.info()
# 8   lat-lon  object 
#data.convert_dtypes().dtypes #podria servir


In [ ]:
print(data.isna().sum())
#price, currency, price_aprox, price_usd tienen la misma cantidad de nulos 20410
#hay que averiguar si price_per_m2 es la relacion de price y surface_total, así podriamos completar nulos en price_per_m2: SI ES

In [ ]:
data.isna().sum()/data.shape[0] *100

#podriamos completar place_name
#cambiar los nombres de las columnas?
#geonames_id está en float?
#lat-long ver si podemos completar donde falte lat y long se podrian borrar las individuales  

In [ ]:

data.describe(include="all")

#count: cuenta el numero de registros no incluye null
#unique:  número de objetos distintos en la columna,
#top: es el dato más frecuente que se produce,
#freq: es la cantidad de veces que aparece el objeto “top” en la columna.

# # de la columna "place_with_parent_name" 
# lat-long hay datos repetidos o son los mismos edificios o representan un area determinada
# operation y country_name 1 solo dato 
# cambiar el dato de geonames_id
# currency tiene 4 tipos de monedas
# hay un tipo de propiedad con 32 habitaciones?
# hay un tipo de propiedad con 3150 pisos?

In [ ]:
#valores unicos x columna
for columnas in data.columns:
    print("")
    print(f'Nombre:{columnas}')
    print(data[columnas].value_counts())

In [ ]:
corr = data.set_index('place_name').corr()
sm.graphics.plot_corr(corr, xnames=list(corr.columns))
plt.show()

#Hay una correlacion entre price y price_aprox_local_currency : price está en dolares y local currency en pesos
#al multiplicar price por la tasa de cambio tenemos price_aprox_local currency

In [ ]:
#vemos el tipo de cambio utilizado
tasa_cambio = data['price_aprox_local_currency'] / data['price_aprox_usd']
#data["tipo_de_cambio"] = data['price_aprox_local_currency'] / data['price_aprox_usd']
#data["tipo_de_cambio"]
tasa_cambio.unique()

#aqui vemos que aparecen datos diferentes por los decimales,entonces vamos a redondear a 2 decimales 
#price_aprox_local_currency     20410
#price_aprox_usd                20410

In [ ]:
print(tasa_cambio.round(2).unique()) #a ver si hay otro valor
print(tasa_cambio.isna().sum()) # 

In [ ]:
data['price_aprox_usd'].sample(10)

In [ ]:
data['price_aprox_usd'].notnull().sum()

In [ ]:
data['price_aprox_usd'].notnull

In [ ]:
#price                          20410
#price_aprox_usd                20410  #price tiene la misma cantidad de registris que price_aprox_usd 
#currency

#surface_total_in_m2            39328
# price_usd_per_m2              52603
# price_per_m2                  33562

buscando_precio_usd_m2 = data['price_aprox_usd'] / data['surface_total_in_m2']
print(buscando_precio_usd_m2)
print(buscando_precio_usd_m2.isna().sum())

#aqui vemos si el dato de price_usd_per_m2 es la relacion de price_aprox entre la superficie_total_in_m2
#price está en USD y otras monedas entonces el calculo va a estar en dolar u otras monedas 
#como la relacion de estas dos columnas me da los datos de precio_usd_per_m2, si tengo nan en alguna de estas columnas lo puedo reemplazar
#hacer una comparacion entre esta columna y la de data

In [ ]:
#agrego la columna nueva a data
data["precio_usd_m2_calculado"]= data['price_aprox_usd'] / data['surface_total_in_m2']

In [ ]:
#comparo las columna original con la columna de precio_m2 original
precio_usd_m2_match = (data['price_usd_per_m2'] == data['precio_usd_m2_calculado'])
precio_usd_m2_match.value_counts()

#

In [ ]:
data["price_usd_per_m2"].isna().sum()



In [ ]:
#price                          20410
#currency                       20411
#price_aprox_local_currency     20410
#price_aprox_usd                20410
#surface_total_in_m2            39328
#surface_covered_in_m2          19907
#price_usd_per_m2               52603
#price_per_m2                   33562

In [ ]:
import seaborn as sns

In [ ]:
#graficamos con un grafico de bigotes como es la presencia de valores extremos en ambas variables 
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
sns.boxplot(data=data_limpia, x="property_type", y='surface_covered_in_m2', ax=axes[0])
sns.boxplot(data=data_limpia, x="property_type", y='surface_total_in_m2', ax=axes[1]);
#efectivamente hay valores extremos

In [ ]:
data.boxplot(column= "rooms", by="property_type") 
data.boxplot(column = "floor", by="property_type")
plt.show()

In [ ]:
# Veamoslo graficamente para una sola variable
#df['variable'].plot(kind = 'box')
#plt.show()

In [ ]:
#para los metros cubiertos y metros totales
#df[['Metros_Cubiertos','Metros_Totales']].plot(kind = 'box', logy = True)
#plt.show()

In [ ]:
print(data["rooms"].info())
print("__________________________")
print("Cantidad de datos nulos: ", data["rooms"].isnull().sum())


In [ ]:
data[['price_aprox_local_currency',"price", "currency"]].describe(include ="all").round(2)
#no muestra mucho solo queria ver

Exploracion de las columnas relacionadas con la ubicacion Por Provincia, Ciudad/Barrio 
- place_name                        
- place_with_parent_names            
- state_name  
--------------------
- Exploramos las columnas 
- Visualizar los nan de estas columnas
- Corroborar que hay relacion entre el barrio/ciudad o "place_name" y la concatenacion de ubicacion en "place_with_parent_names" para ver si efectivamente corresponden o hay barrios mezclados

- Exploramos las columnas relacionadas a la localizacion politica de los inmuebles

In [ ]:
data[["place_with_parent_names", "state_name", "place_name"]].describe()

In [ ]:
#ubicacion por provincia
data["state_name"].value_counts() #Excludes NA values by default.

#notamos que los datos estan concentrados en Buenos Aires: capital federal, GBA Norte y sur, Cordoba, Santa fe, buenos aires costa atlantica 

In [ ]:
#ubicacion por ciudad-partido o barrio/localidad
 
data["place_name"].value_counts().head(20)

#este atributo tiene contenido ciudades principales/partidos y barrios
#Capital federal es un registro de state_name por lo cual no broinda mayor informacion  
#Tigre es una ciudad al norte de la ciudad de Buenos Aires.
# Nordelta es una localidad urbana en el Partido de Tigre, Provincia de Buenos Aires 

#Capital Federal    1297

In [ ]:
#hacemos un agrupamiento de los place_name que conforman state_name
pd.options.display.max_rows = None
data.groupby(["state_name"])["place_name"].value_counts()

In [ ]:
#Para state_name vemos: cuales son los datos de place_name más frecuente, su frecuencia y cuantos refistros diferentes existen 

data.groupby(["state_name"])["place_name"].describe()

#el atributo place_name está representado principalmente por una ciudad-partido de la provincia, también por barrios como en el
#caso de belgrano en capital federal

#count: cuenta el numero de registros no incluye null
#unique:  número de objetos distintos en la columna,
#top: es el dato más frecuente que se produce,
#freq: es la cantidad de veces que aparece el objeto “top” en la columna

In [ ]:
#Para el atribut place_with_parent_names vemos algunos registros ya que vimos en la descripcion que más de 1100 registros unicos 

data["place_with_parent_names"].value_counts().head(20) 

#el atributo concatena las divisiones de la localizacion: pais/ "country_name", la provincia/ "state_name" (o division de la provincia),
#  la ciudad principal/partido-municipio o barrio en el caso de capital federal"place_name"
#y un adicional que puede ser una localidad del partido (ejem: |Argentina|Bs.As. G.B.A. Zona Norte|Tigre|*Nordelta*|)
#el registro tiene entre 2  y 4 concatenaciones: 2 concantenaciones solo muestra hasta la ubicacion por provincia, no brinda informacion relevante

###|Argentina|Capital Federal|  1297 no tiene nombre de barrio, desestimar o completar? 
#|Argentina|Córdoba|    2648

#esta columna puede usarse para completar los datos faltantes de place_name 
#podemos chequear si el atributo "place_name" coincide con los datos aquí plasmados

- Visualizar los nan de las columnas

In [ ]:
# de la aplicacion del método isna().sum() en data obtuvimos la cantidad de nulos por atributo de localizacion 
# place_name                        23
# place_with_parent_names            0
# country_name                       0
# state_name                         0

#solo la columna place_name tiene valores nulos

In [ ]:
#vemos que filas son y como podemos completarlas o no

data[data.place_name.isna()].head(1)

#Podemos ver que todos los registros en  de place_with_parent.. tienen un dato que no fue completado y aparece entre dos ||
#al ser 23 datos se pueden ubicar los barrios usando las coordenadas o pueden completarse con fillna con el dato de "Tigre"
#  ya que vimos en place_name que "Tigre" tiene 3324 registros, un valor signif

#mas adelante identificaremos que filas de place_name son nan

Visualicemos si existe relacion entre las tres columnas de ubicacion politica del dataframe

In [ ]:
data.loc[:, ["place_with_parent_names", 'state_name', 'place_name']].sample(20)
#aparentemente parecen coincidir

In [ ]:
print(data["place_with_parent_names"][4]) #ejemplo de 4 divisiones
print(data["place_with_parent_names"][121216])
print(data["place_with_parent_names"][43904]) #ejemplo de 3 divisiones
print(data["place_with_parent_names"][78457]) 
print(data["place_with_parent_names"][87750]) #ejemplo de 2 divisiones

#el registro con 2 concantenaciones solo muestra hasta la ubicacion por provincia, no brinda informacion relevante


- Corroborar que hay relacion entre place_name (ciudad/partido/barrio)  y los registros completados en "place_with_parent_names" o no

"place_with_parent_names" es una concatenacion de |país|provincia|ciudad-partido|localidad o barrio| 

"place_name" está representado principalmente por una ciudad-partido de la provincia y en algunos casos la localidad o el barrio

De este modo, para ver la relacion de ambas columnas podemos separar en bloques con "|" y extraer el str asociado a la ciudad-partido y localidad o barrio. 
para que segun la los valores unicos de place_name es el ##tercer## str del bloque

vemos que hay registros que solo se completó hasta la provincia como |Argentina|Córdoba|, hay que investigar un poquito si es porque la ciudad es la misma o en el caso de
|Argentina|Capital Federal, donde vimos que en place_name se completa de nuevo con capital federal

In [ ]:
#con la funcion split separamos el string completo
data['place_with_parent_names'].apply(lambda x: len(x.split("|"))) 

#veo como realiza la separaciones: toma el simbolo | como una separacion al principio y al final 
# 0 |Argentina|Capital Federal|Mataderos| tiene 5 sep  
# 4 |Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro| tiene 6 sep

In [ ]:
#probando
holis = data['place_with_parent_names'].apply(lambda x: x.lstrip("|").rstrip("|").split("|"))
holis.apply(lambda x: len(x)).value_counts()

#aqui agrupamos la division de str por | 
# |Arg|buenos aires|tigre|nordelta||
# y el orden de frecuencia de la cantidad str
# 3 str "Arg","buenos aires", "tigre"
# 4 str "Arg","buenos aires", "tigre", "nordelta"
# 2 str "Arg","buenos aires"
# 5 str "Arg","buenos aires", "tigre", "nordelta", "x" o "" ¿?

In [ ]:
# Contamos la cantidad de grupos de str de cada fila de la lista place_with_parent_names
##tengo diferencia con Luis en 5 y 6
data['place_with_parent_names'].apply(lambda x: len(x.split("|"))).value_counts()

#aqui agrupamos el conteo de separaciones de
# |Arg|buenos aires|tigre|nordelta||
# y el orden de frecuencia
# 4 divisiones  |Arg|buenos aires|
#5 divisiones |Arg|buenos aires|tigre|
#6 divisiones |Arg|buenos aires|tigre|nordelta|
#7 divisiones |Arg|buenos aires|tigre|nordelta|x| debe existir otro dato x o doble ||

In [ ]:
#vemos cuantas separaciones unicas existen
data['place_with_parent_names'].apply(lambda x: len(x.split("|"))).unique()

In [ ]:
#separo el data para mejor manejo las columnas de interes ##ya lo habia hecho arriba
df_loc_politica_only = data.loc[:, ["place_with_parent_names", "state_name", "place_name"]]
df_loc_politica_only.describe()

|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|
    1               2                           3          4
          | state_place                | place_name  |place_name|(alternativo barrio)

In [ ]:
##place_with_parent_names vs state_place
##para ver la coincidencia con la provincia me interesa el 2do str de la cadena place_with_parent y creamos una nueva columna
sep_str_1 = 2 #esta variable es la posicion que ocupa provincia en el str de place_with_parent_names
df_loc_politica_only['state_filt'] = df_loc_politica_only["place_with_parent_names"].apply( lambda x: x.split("|")[sep_str_1])
df_loc_politica_only['state_filt'] #creamos una nueva columna con los datos extraidos de place_with_parent...

In [ ]:
#comparo si el filtro de state "state_filt" es igual a la columna original "state_place"
son_iguales_1 = df_loc_politica_only["state_name"] == df_loc_politica_only["state_filt"]
son_iguales_1.value_counts()

In [ ]:
# country_name (de luis)
print("null values:", df_loc_politica_only['state_name'].isna().sum())
mask = ~df_loc_politica_only.apply(lambda x: x['state_name'] in x['place_with_parent_names'], axis=1)
print("place_with_parent_names difference with state_name:", df_loc_politica_only[mask].shape[0])

In [ ]:
#no me quiere resolver
# mask_5 = location_data['place_with_parent_names'].apply(lambda x: len(x) == 5)
# location_data_5_elements = location_data[mask_5].copy()
# location_data_5_elements['place_with_parent_names'][17]

In [ ]:
print(df_loc_politica_only.iloc[17,0])
print(df_loc_politica_only.iloc[17,1])
print(df_loc_politica_only.iloc[17,2])

In [ ]:
#place_with_parent_names vs place_name
##para ver la coincidencia con ciudad/partido me interesa el 3er str que ocupa la cadena place_with_parent..
# Al comparar con place_name pueden pasar dos cosas: 1) que no esté el dato de ciudad/partido y por consiguiente tampoco
# el barrio (en ambos casos aqui dara celdas vacias)
#2) que este completo y además tenga la localidad/barrio, pero el lambda no me da ese str porque hay casos donde no esta
# ese dato y no puede correr el código (por lo que place_name debe coincidir con alguna de ellas dos ¿como lo hago?)

sep_str_2 = 3 #extrae el caso 1)
df_loc_politica_only['place_filt'] = df_loc_politica_only["place_with_parent_names"].apply( lambda x: x.split("|")[sep_str_2])
df_loc_politica_only['place_filt'].value_counts()

In [ ]:
df_loc_politica_only.iloc[17,:] ##doy una vista previa a la data, veo que hay algunas celdas vacias y otras que no coinciden

In [ ]:
#comparo las columnas
places_match = (df_loc_politica_only['place_filt'] == df_loc_politica_only['place_name'])
places_match.value_counts()

#hay 35193 registros donde place_name no coincide con la posicion de place_with_parent_name, esto es porque esta ultima columna
# no está completa y porque no he sacado los datos de la cuarta posicion

In [ ]:
#lo mismo que arriba pero las que son verdadera para sacar los indices
place_no_match = (df_loc_politica_only['place_filt'] != df_loc_politica_only['place_name'])
df_loc_politica_only_no_match = df_loc_politica_only[place_no_match]
df_loc_politica_only_no_match.index

In [ ]:
print(place_no_match.value_counts())

In [ ]:
#quiero ver si place_filt tiene valores nulos
mascara_nulos_place_filt= df_loc_politica_only.iloc[:,3].astype(str) == "nan"
print(mascara_nulos_place_filt.value_counts())

In [ ]:
#en la nueva columna place_filt creo una mascara para ver las celdas con registros vacios: donde place_with_parent.. no tienen localidad/barrio)
#saco el total de casos y el index
mascara_place_filt_vacios = df_loc_politica_only["place_filt"] == ""

print(mascara_place_filt_vacios.value_counts())

#print(df_loc_politica_only[mascara_vacios])
print(df_loc_politica_only[mascara_place_filt_vacios].index)

#4780 datos vacios que le faltan completar el partido/barrio:en la columna original de place_name estos datos están compuestos
# por el registro de state_place, por lo que dejarlos así no brindarian informacion a una relacion por localida-barrio 

In [ ]:
mascara_nulos = df_loc_politica_only["place_name"].astype(str) == "nan"
#mascara_nulos= df_loc_politica_only.iloc[:,2].astype(str) == "nan"

#mascar_nulos

print(mascara_nulos.value_counts())
print(df_loc_politica_only[mascara_nulos].index)

In [ ]:
place_no_match = (data_copy['place_filt_2'] != data_copy['place_name'])
data_copy_no_match = data_copy[place_no_match]
data_copy_no_match.loc[:, ["place_with_parent_names", 'place_name', 'place_filt_2']]

#hay 35193 filas diferentes entre place_name y place_filt_2 pero posiblemnete correctas donde no he podido comparar con el 4 str y 
# contiene los datos vacios

In [ ]:
#no borrar

#sacamos los de 4 separaciones 
data_copy = data
#split_place_filtro = data_copy['place_with_parent_names'].apply(lambda x: len(x.split("|"))!=4)
#data_copy = data_copy.loc[split_place_filtro]

In [ ]:
#extraemos la posicion donde está barrio y lo agregamos en una nueva columna >> place_filt.
#filtramos el ##tercer## str del bloque
sep_str = 3
data_copy['place_filt_2'] = data_copy['place_with_parent_names'].apply( lambda x: x.split("|")[sep_str])

#esta no me sirve para comparar diferencias porque en realidad no puedo extraer la cuarta columna por eso me dan diferentes

In [ ]:
data_copy.loc[:, ["place_with_parent_names", 'place_name', 'place_filt_2']].sample(20)

In [ ]:
print(data["place_with_parent_names"][94288]) #place_name tiene tanto nombre de ciudades como de barrios
print(data["place_with_parent_names"][66350])
print(data["place_with_parent_names"][97449])
print(data["place_with_parent_names"][21596]) 

In [ ]:

place_no_match = (data_copy['place_filt_2'] != data_copy['place_name'])
data_copy_no_match = data_copy[place_no_match]
data_copy_no_match.loc[:, ["place_with_parent_names", 'place_name', 'place_filt_2']]

#hay 35193 filas diferentes entre place_name y place_filt_2 pero posiblemnete correctas donde no he podido comparar con el 4 str y 
# contiene los datos vacios

In [ ]:
data_copy_no_match.index #aqui estan todos los indices de las filas que no coinciden, si quiero volver a matchear place_nam de los barrios
# elimino los indices donde están los vaciis

In [ ]:
pruebo = data_copy_no_match["place_filt"] == "" #si contiene los vacios
pruebo.unique()

In [ ]:
pruebo.value_counts()

In [ ]:
#de la data original tomo la columna place_name
extr_place_name = data.iloc[:,2].astype(str) #no me lee los nan sino lo convierto a str

extr_place_name

In [ ]:
masc = extr_place_name == "nan" #creo una mascar para saber que filas son nan
#masc

In [ ]:
filas_nan = extr_place_name[masc].index #aplico la masc y creo una variable con los indices de las filas nan
filas_nan

In [ ]:
data.loc[filas_nan, ["place_with_parent_names", 'state_name', 'place_name']] #visualizo los nan para ver como completarlos


In [ ]:
import re

In [ ]:
texto_tigre = data.loc[filas_nan, ["place_with_parent_names"]].astype(str)
texto_tigre
#re.findall(r'Tigre', texto_tigre)

In [ ]:
import re  

In [ ]:
data.iloc[121219:,]
#podriamos comparar las columnas a ver si pasa esto de que no hay datos geor y 
#esta fila sirve si la posterior correlacion la hago con state_name/provincia

In [ ]:
data[["description", "title"]][121219:]

In [ ]:
data[data.surface_total_in_m2.isna()].head(4)


In [ ]:
print(data["description"][121219])
print("----------")
print(data["title"][121219])

In [ ]:
print(data["description"][3])
print("----------")
print(data["title"][3])

In [ ]:
#probar con esto

In [ ]:

#ejemplo de pagina
#df.apply(lambda x: x['Name'].split('|', 1)[-1] if x['Age'] > 16 and len(x['Name'].split('|',1))>1 else np.nan, axis=1)

VAMOS CON LOS DATOS DE UBICACION GEOGRAFICA
1. # grupos de columas para poder trabajar con ellas de forma mas sencilla
places = ['place_name','place_with_parent_names', 'country_name','state_name']
geolocation = ['geonames_id', 'lat-lon','lat','lon']

# Creamos un nuevo dataframe con las columnas que me interesan
geo_location_data = data[ geolocation].copy()
geo_location_data.head()

In [ ]:
# grupos de columas para poder trabajar con ellas de forma mas sencilla
places = ['place_name','place_with_parent_names', 'country_name','state_name']
geolocation = ['state_name','geonames_id', 'lat-lon','lat','lon']

# Creamos un nuevo dataframe con las columnas que me interesan
geo_location_data = data[ geolocation]
geo_location_data.head()

In [ ]:
geo_location_data.loc[:, ["geonames_id", "lat", 'lon', 'state_name']].head(15)


In [ ]:
mascara_nulos = df_loc_politica_only["place_name"].astype(str) == "nan"

In [ ]:
#creo un nuevo dataframe con las columnas 

In [ ]:

#filas_geo_nans = geo_columnas[masc_geo].index
#filas_geo_nans
